In [1]:
import numpy as np
import math

pk = np.linspace(0, 1, 21)
pk
sample = np.random.normal(0, 1, 1000)

In [6]:
qk = []
for p in pk:
    n_cal = len(sample)
    quantile = np.quantile(sample, p) 
    qk.append(quantile)

In [7]:
pk

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

In [8]:
qk

[-2.8523231877040853,
 -1.5519606435837998,
 -1.247509418280673,
 -1.024120510012469,
 -0.8542779476925743,
 -0.7003892779991409,
 -0.5095729603692519,
 -0.36642139343263186,
 -0.2404032705782276,
 -0.1074975472535216,
 0.01872928102705225,
 0.1627870698909281,
 0.30727690546149355,
 0.4182671654641155,
 0.5540319881409299,
 0.6863722358038725,
 0.8442442265507587,
 1.0733032887762557,
 1.3422020370389995,
 1.712185365519452,
 3.1055792499101207]

In [ ]:
intervals = []
density = []

for i in range(1, len(qk)):
    interval = (qk[i-1], qk[i])
    intervals.append(interval)
    weight = pk[i] - pk[i-1]
    prob =  weight / (qk[i] - qk[i - 1])
    density.append(prob)  

width = 1 / len(intervals)  
width

In [ ]:
intervals

In [ ]:
# mean
cum_ = 0.0
for interval, prob in zip(intervals, density):
    cum_ = (interval[1] - interval[0]) / 2 * prob
cum_

In [ ]:
# cdf
def cdf(x):
    if x < min(qk):
        return 0
    elif x > max(qk):
        return 1

    for i, interval in enumerate(intervals):
        left, right = interval                   
        if x < right and x >= left:
            # cumulated weights of previous intervals
            cum_weight = pk[i]
            # plus the cumulated weights at the current interval
            cum_weight += (x - left) / (right - left)  *0.05
            return cum_weight


In [ ]:
cdf(-3.)

In [ ]:
# rvs
def rvs(size=1):
    def _sample_single_sample():
        # first sample an interval
        interval_idx = np.random.randint(0, len(intervals), 1)[0]
        interval = intervals[interval_idx]
        # inside this interval, sample uniformly
        return np.random.uniform(interval[0], interval[1], size=1)[0]
        
    samples = []
    for i in range(size):
        samples.append(_sample_single_sample())

    assert len(samples) == size
    return np.array(samples)

In [ ]:
def ppf(q) -> float:
    if q > 1 or q < 0:
        return np.nan
    
    for i, percentile in enumerate(pk):
        if q <= percentile:
            interval = intervals[i-1]
            left, right = interval
            break
    
    ratio = (q - percentile) / width 
    return ratio * (right - left) + left

In [ ]:
middles = []

for interval in intervals:
    left, right = interval
    middles.append((left + right) / 2)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(middles, density)